In [1]:
import pandas as pd
import numpy as np
import datetime
import sys
sys.path.append("/Users/derekdewald/Documents/Python/Github_Repo/d_py_functions/")


In [8]:
v1 ='/Users/derekdewald/Documents/Python/Github_Repo/d_py_functions/'
d1 = ReadDirectory(v1)
d1

['TextFunctions.py',
 '.DS_Store',
 'FeatureEngineering.py',
 'Archive',
 'DateFunctions.py',
 'Organization.py',
 'D_Python_Functions_25-Aug-25.xlsx',
 'Connections.py',
 'UtilityFunctions.py',
 '~$D_Python_Functions_25-Aug-25.xlsx',
 'DataSets.py',
 '__init__.py',
 'SharedFolder.py',
 '__pycache__',
 'Visualization.py',
 'V2',
 'EDA.py',
 'TimeSeries_WIP.py',
 'OtherFunctions.py',
 'DFProcessing.py',
 'ArchivedFunctions.py',
 'MLPipeline.py',
 '.ipynb_checkpoints',
 'SQL.py',
 'D_Python_Functions.xlsx',
 'Validation.py']

In [5]:
from SharedFolder import ReadDirectory

v2 = '/Users/derekdewald/Documents/Python/Github_Repo/d_py_functions/v2'
d2 = ReadDirectory(v2)
d2

['FeatureEngineering.py',
 'DateFunctions.py',
 'DFStatistical.py',
 'Organization.py',
 'DictionaryManipulation.py',
 'Connections.py',
 'UtilityFunctions.py',
 'SharedFolder.py',
 'Visualization.py',
 'D_Testing.py',
 'EDA.py',
 'OtherFunctions.py',
 'DataMaps.py',
 'DFProcessing.py',
 'Test_Review_Implement.txt',
 'MLPipeline.py',
 'SQL.py',
 'ListManipulation.py',
 'Validation.py',
 'TimeSeries.py']

In [3]:
from TextFunctions import ReadPythonFiles
ReadPythonFiles(file_list=['Connections.py'])

,Description,Arguments,Return,Code,File
DownloadFilesFromGit,Function to Download Files from Github to a de...,"[User:, Repo:, folder:, output_folder:, ]",,"def DownloadFilesFromGit(user='derek-dewald',\...",Connections.py
ParamterMapping,"Function to Google Mapping Sheet, which is use...",[Definition (Str): Key word used to Access ind...,,"def ParamterMapping(Definition=""""):\n \n ...",Connections.py
BackUpGoogleSheets,Function to Create a Backup of Information Sto...,"[None, ]",,def BackUpGoogleSheets(location='/Users/derekd...,Connections.py
GoogleProcessSheetLinks,"Function to Google Mapping Sheet, Navigate to ...","[, ]",,def GoogleProcessSheetLinks():\n \n '''...,Connections.py
NavigateUsingDMap,"Function to Google Mapping Sheet, Navigate to ...","[, ]",,def NavigateUsingDMap():\n \n '''\n ...,Connections.py
CreateTableAnalytics,Function utilized to Run Script Created from F...,"[sql(text): SQL Text, generated primarily excl...",,def CreateTableAnalytics(sql):\n '''\n F...,Connections.py
DeleteAnalyticalTable,No description available,"[table_name, condition, by_pass_validation, re...",None,"def DeleteAnalyticalTable(table_name,\n ...",Connections.py
IterativeTestInsertColumnByColumn,Attempts to insert each column from the first ...,"[df, table_name, total_rows]",None,"def IterativeTestInsertColumnByColumn(df, tabl...",Connections.py
UpdateAnalyticsTable,Function to insert data into a SQL Server tabl...,"[df, table_name, batch_size]",None,"def UpdateAnalyticsTable(df, table_name, batch...",Connections.py
INSERT_STATEMENT,Function Used to Insert Data Into a Table in M...,"[df, table_name]",None,"def INSERT_STATEMENT(df,table_name):\n \n ...",Connections.py


In [7]:
from TextFunctions import ReadPythonFiles
ReadPythonFiles(location='/Users/derekdewald/Documents/Python/Github_Repo/d_py_functions/V2/',file_list=['Connections.py'])

,Description,Arguments,Return,Code,File
DownloadFilesFromGit,Function to Download Files from Github to a de...,"[User:, Repo:, folder:, output_folder:, ]",,"def DownloadFilesFromGit(user='derek-dewald',\...",Connections.py
ParamterMapping,"Function to Google Mapping Sheet, which is use...",[Definition (Str): Key word used to Access ind...,,"def ParamterMapping(Definition=""""):\n \n ...",Connections.py
BackUpGoogleSheets,Function to Create a Backup of Information Sto...,"[None, ]",,def BackUpGoogleSheets(location='/Users/derekd...,Connections.py
NavigateUsingDMap,"Function to Google Mapping Sheet, Navigate to ...","[, ]",,def NavigateUsingDMap():\n \n '''\n ...,Connections.py


['TextFunctions.py',
 '.DS_Store',
 'FeatureEngineering.py',
 'Archive',
 'DateFunctions.py',
 'Organization.py',
 'D_Python_Functions_25-Aug-25.xlsx',
 'Connections.py',
 'UtilityFunctions.py',
 '~$D_Python_Functions_25-Aug-25.xlsx',
 'DataSets.py',
 '__init__.py',
 'SharedFolder.py',
 '__pycache__',
 'Visualization.py',
 'V2',
 'EDA.py',
 'TimeSeries_WIP.py',
 'OtherFunctions.py',
 'DFProcessing.py',
 'ArchivedFunctions.py',
 'MLPipeline.py',
 '.ipynb_checkpoints',
 'SQL.py',
 'D_Python_Functions.xlsx',
 'Validation.py']

In [28]:
def CompareFunctions(func1,func2):
    '''
    Function to Compare 2 Functions, looking for the first point of differences.

    Parameters:


    Returns:


    Date Created: August 17, 2025
    Date Last Modified:


    '''

    list1 = FunctionToSTR(func1)
    list2 = FunctionToSTR(func2)

    for record in range(max(len(list1),len(list2))):
        if list1[record]==list2[record]:
            continue
        else:
    
            print(f"Shared Text:\n{' '.join(list1[record-10:record])}")
    
            print(f'Record 1\n')
            print(" ".join(list1[record:record+25]))
            
            print(f'Record 2\n')
            print(" ".join(list2[record:record+25]))
            break

            

Shared Text:
with respect to 0 values. Is processed after exclude_blanks, as
Record 1

such it can include both blanks and true 0 values. ''' temp_dict = {} is_numeric = pd.api.types.is_numeric_dtype(df[column_name]) if is_numeric: temp_dict['SUM'] = df[column_name].sum() temp_dict['MEAN'] = df[column_name].mean()
Record 2

''' temp_dict = {} is_numeric = pd.api.types.is_numeric_dtype(df[column_name]) if is_numeric: temp_dict['SUM'] = df[column_name].sum() temp_dict['MEAN'] = df[column_name].mean() temp_dict['STD_DEV'] = df[column_name].std() temp_dict['MEDIAN'] = df[column_name].median() temp_dict['MAX'] = df[column_name].max() temp_dict['MIN']
